# Volume 2, Chapter 13: Network Documentation Basics

**Auto-generate always-accurate documentation from network configs**

From: AI for Networking Engineers - Volume 2, Chapter 13

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/main/CODE/Colab-Notebooks/Vol2_Ch13_Network_Documentation.ipynb)

## 🎯 What You'll Learn

- Auto-generate documentation from device configs
- Create network topology diagrams with AI
- Build a documentation pipeline that stays in sync

**The Problem**: Documentation scattered across SharePoint (outdated), wikis (conflicting), tribal knowledge (gone when Bob leaves).

**The Solution**: Generate docs directly from configs - always accurate because it's from the source of truth.

## 🔧 Setup

In [ ]:
# Install required packages
!pip install -q anthropic

In [ ]:
# Set API keys
import os
from getpass import getpass

if 'ANTHROPIC_API_KEY' not in os.environ:
    os.environ['ANTHROPIC_API_KEY'] = getpass('Enter Anthropic API key: ')

print("✓ API key set!")

## 📖 Demo 1: Generate Device Documentation from Config

In [ ]:
from anthropic import Anthropic
import json
from datetime import datetime

# Initialize client
client = Anthropic()

# Sample router configuration
sample_config = """
hostname router-core-01
!
interface Loopback0
 ip address 192.168.1.1 255.255.255.255
!
interface GigabitEthernet0/0
 description Uplink to ISP
 ip address 203.0.113.1 255.255.255.252
 no shutdown
!
interface GigabitEthernet0/1
 description Connection to Datacenter
 ip address 10.0.1.1 255.255.255.0
 no shutdown
!
router ospf 1
 router-id 192.168.1.1
 network 10.0.0.0 0.0.255.255 area 0
 network 192.168.1.1 0.0.0.0 area 0
!
router bgp 65001
 bgp router-id 192.168.1.1
 neighbor 203.0.113.2 remote-as 65002
 neighbor 203.0.113.2 description ISP_PRIMARY
!
ip access-list extended MANAGEMENT_ACCESS
 permit tcp 10.0.0.0 0.0.255.255 any eq 22
 deny ip any any log
!
line vty 0 4
 access-class MANAGEMENT_ACCESS in
 transport input ssh
"""

print("Sample config loaded:")
print(sample_config[:300] + "...")

In [ ]:
def generate_device_overview(config: str, hostname: str) -> dict:
    """Generate high-level device documentation."""
    
    prompt = f"""Analyze this network device configuration and create documentation.

Device: {hostname}
Configuration:
{config}

Extract and document:
1. Device role (core router, access switch, firewall, etc.)
2. Management IP address
3. Routing protocols in use (BGP, OSPF, EIGRP, static)
4. Key features enabled (HSRP, VRF, QoS, etc.)
5. Interface count and types
6. Notable configurations or policies

Return as JSON:
{{
    "hostname": "device name",
    "role": "device role",
    "management_ip": "IP address",
    "routing_protocols": ["list of protocols"],
    "key_features": ["list of features"],
    "interface_summary": "summary of interfaces",
    "notable_config": "anything important to know"
}}

JSON:"""

    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1500,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )
    
    doc_data = json.loads(response.content[0].text)
    doc_data['generated_at'] = datetime.now().isoformat()
    
    return doc_data

# Generate overview
print("Generating device overview...")
overview = generate_device_overview(sample_config, "router-core-01")

print("\n" + "="*60)
print("DEVICE OVERVIEW")
print("="*60)
print(json.dumps(overview, indent=2))

In [ ]:
def generate_interface_table(config: str) -> str:
    """Generate markdown table of all interfaces."""
    
    prompt = f"""Extract all interfaces from this config and create a markdown table.

Config:
{config}

Create a table with columns:
| Interface | IP Address | Status | Description | VLAN/VRF |

Include ALL interfaces (physical, loopback, tunnel, etc.)

Markdown table:"""

    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=2000,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.content[0].text

# Generate interface table
print("Generating interface table...")
interface_table = generate_interface_table(sample_config)

print("\n" + "="*60)
print("INTERFACE TABLE")
print("="*60)
print(interface_table)

In [ ]:
def generate_routing_documentation(config: str) -> str:
    """Document routing configuration."""
    
    prompt = f"""Document the routing configuration from this device.

Config:
{config}

Create documentation covering:

## Routing Protocols
- Which protocols are enabled
- Process IDs, AS numbers
- Router IDs

## Static Routes
- Destination networks
- Next hops
- Purpose

## Route Redistribution
- What's redistributed where
- Filters applied

## Routing Policies
- Route-maps
- Prefix-lists
- Access-lists affecting routing

Format as markdown with sections and bullet points.

Documentation:"""

    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=2500,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.content[0].text

# Generate routing docs
print("Generating routing documentation...")
routing_docs = generate_routing_documentation(sample_config)

print("\n" + "="*60)
print("ROUTING DOCUMENTATION")
print("="*60)
print(routing_docs)

## 📖 Demo 2: Create Network Topology Diagrams

In [ ]:
# Sample CDP/LLDP output
cdp_output = """
Device ID: switch-dist-01
Interface: GigabitEthernet0/1, Port ID: GigabitEthernet1/0/1
Platform: cisco WS-C3850

Device ID: router-core-02
Interface: GigabitEthernet0/2, Port ID: GigabitEthernet0/2
Platform: Cisco 4451-X

Device ID: firewall-01
Interface: GigabitEthernet0/3, Port ID: Ethernet1/1
Platform: Palo Alto PA-5250
"""

print("Sample CDP output loaded")

In [ ]:
def extract_neighbors_and_generate_diagram(cdp_output: str, local_device: str) -> str:
    """Extract neighbors and generate Mermaid diagram."""
    
    prompt = f"""Analyze this CDP/LLDP output and create a network topology diagram.

Local Device: {local_device}
CDP Output:
{cdp_output}

Generate a Mermaid flowchart diagram showing:
1. All devices as nodes
2. All connections between them
3. Interface labels on the links
4. Use appropriate shapes:
   - Rectangle for routers
   - Cylinder/database shape for switches
   - Hexagon for firewalls

Return ONLY the Mermaid syntax, starting with ```mermaid and ending with ```

Mermaid diagram:"""

    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1500,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.content[0].text

# Generate diagram
print("Generating topology diagram...")
diagram = extract_neighbors_and_generate_diagram(cdp_output, "router-core-01")

print("\n" + "="*60)
print("TOPOLOGY DIAGRAM (Mermaid)")
print("="*60)
print(diagram)
print("\n💡 Copy this into any Markdown viewer that supports Mermaid (GitHub, Notion, etc.)")

## 📖 Demo 3: Complete Documentation Generator

In [ ]:
def generate_complete_documentation(config: str, hostname: str) -> str:
    """Generate complete device documentation."""
    
    print(f"Generating complete documentation for {hostname}...")
    
    # Get all sections
    print("  → Device overview...")
    overview = generate_device_overview(config, hostname)
    
    print("  → Interface table...")
    interfaces = generate_interface_table(config)
    
    print("  → Routing documentation...")
    routing = generate_routing_documentation(config)
    
    # Build complete doc
    doc = f"""# {hostname} - Device Documentation

**Generated**: {overview['generated_at']}
**Device Role**: {overview['role']}
**Management IP**: {overview['management_ip']}

---

## Overview

**Routing Protocols**: {', '.join(overview['routing_protocols'])}
**Key Features**: {', '.join(overview['key_features'])}

{overview['notable_config']}

---

## Interfaces

{interfaces}

---

## Routing Configuration

{routing}

---

*This documentation is auto-generated from device configuration.*
"""
    
    print("  ✓ Complete!")
    return doc

# Generate full documentation
full_doc = generate_complete_documentation(sample_config, "router-core-01")

print("\n" + "="*60)
print("COMPLETE DOCUMENTATION")
print("="*60)
print(full_doc)

## 🧪 Try Your Own Config!

In [ ]:
# Paste your own config here
your_config = """
# Paste your device config here
# Example:
# hostname my-switch
# interface Vlan10
#  ip address 10.10.10.1 255.255.255.0
"""

your_hostname = input("Enter device hostname: ") or "my-device"

if len(your_config.strip()) > 50:  # Has real content
    your_doc = generate_complete_documentation(your_config, your_hostname)
    print("\n" + your_doc)
else:
    print("\n⚠️ Paste your config in the your_config variable above and run again!")

## 🎯 Key Takeaways

### Why This Matters
- **Problem**: Docs drift from reality → incidents, slow onboarding
- **Solution**: Generate docs from configs → always accurate

### What We Built
1. **Device Overview** - Role, IPs, protocols, features
2. **Interface Table** - All interfaces in markdown format
3. **Routing Documentation** - Protocols, neighbors, policies
4. **Topology Diagrams** - Mermaid diagrams from CDP/LLDP

### Production Tips
- **Automate**: Run on schedule (daily/weekly) or on config change
- **Version Control**: Store docs in Git to track changes
- **CI/CD Integration**: Generate docs in your pipeline
- **Cost Control**: Use Haiku for simple extraction, Sonnet for complex analysis

### ⚠️ Watch Out For
- Sensitive data (passwords, keys) → sanitize configs first
- API costs at scale → batch updates, cache results
- Stale configs → fetch live configs from devices

## 📚 Next Steps

- **Chapter 14**: RAG Fundamentals - Make this documentation searchable with AI
- Add your own configs and build a documentation library
- Integrate with your CI/CD pipeline